## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [119]:
import re

import numpy as np
import pandas as pd
import scipy.stats as stats

nhl_df = pd.read_csv("assets/nhl.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities_cpy = cities.copy()
cnt = 0


def nhl_decomp(e):
    global cnt
    global cities_cpy
    teams = re.findall(
        "(Maple \w+|Blue \w+|White \w+|Red \w+|Golden \w+|(?!Maple |Blue |Red |White |Golden )\w+)",
        e.loc["NHL"],
    )
    if not teams:
        teams = [""]
    for team in teams:
        try:
            cities_cpy.iloc[cnt] = e
        except:
            cities_cpy = cities_cpy.append(e, ignore_index=True)
        cities_cpy.iloc[cnt].loc["NHL"] = team
        cnt += 1


def nhl_correlation():
    global nhl_df
    global cities_cpy
    global cities
    global cnt
    cities_cpy = cities.copy()
    nhl_cpy = nhl_df[nhl_df["year"] == 2018].copy()
    nhl_cpy.drop(index=[0, 9, 18, 26], inplace=True)
    nhl_cpy["WL%"] = nhl_cpy["W"].astype("float") / (
        nhl_cpy["W"].astype("float") + nhl_cpy["L"].astype("float")
    )
    nhl_cpy["team"] = nhl_cpy["team"].str.replace("\*$", "", regex=True)
    # area = nhl_cpy["team"].str.extract('^(?!Los|San|St\.|New )(\w*)|(New \w*|St\. \w*|San \w*|Los \w*)')
    team = (
        nhl_cpy["team"]
        .str.extract("((?:Los |San |St\. |New )?(?:\w* )(?:Bay )?(?P<NHL>.*$))")
        .fillna("")
    )
    nhl_cpy["NHL"] = team["NHL"]
    cnt = 0
    cities["NHL"] = cities["NHL"].replace("\[note \d{1,2}\]$", "", regex=True)
    cities.apply(nhl_decomp, axis=1)
    cities_cpy["NHL"] = cities_cpy["NHL"].replace("^—$|^$", np.nan, regex=True)
    cities_cpy.dropna(inplace=True)
    cities_cpy["Metropolitan area"] = cities_cpy["Metropolitan area"].replace(
        " Area$", "", regex=True
    )
    cities_nhl = cities_cpy[
        ["Metropolitan area", "Population (2016 est.)[8]", "NHL"]
    ].merge(nhl_cpy[["NHL", "WL%"]], on="NHL", how="left")
    wl_ratio = cities_nhl.groupby("Metropolitan area", sort=False).agg("mean")["WL%"]

    population_by_region = pd.unique(cities_nhl["Population (2016 est.)[8]"]).astype(
        "float"
    )  # pass in metropolitan area population from cities
    win_loss_by_region = wl_ratio.astype(
        "float"
    )  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(
        win_loss_by_region
    ), "Q1: Your lists must be the same length"
    assert (
        len(population_by_region) == 28
    ), "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [120]:
import re

import numpy as np
import pandas as pd
import scipy.stats as stats

nba_df = pd.read_csv("assets/nba.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities_cpy = cities.copy()
cnt = 0


def nba_decomp(e):
    global cnt
    global cities_cpy
    teams = re.findall(
        "(Maple \w+|Blue \w+|White \w+|Red \w+|Golden \w+|Trail \w+|(?!Maple |Blue |Red |White |Golden |Trail )\w+)",
        e.loc["NBA"],
    )
    if not teams:
        teams = [""]
    for team in teams:
        try:
            cities_cpy.iloc[cnt] = e
        except:
            cities_cpy = cities_cpy.append(e, ignore_index=True)
        cities_cpy.iloc[cnt].loc["NBA"] = team
        cnt += 1


def nba_correlation():
    global nba_df
    global cities_cpy
    global cnt
    cities_cpy = cities.copy()
    nba_cpy = nba_df[nba_df["year"] == 2018].copy()
    nba_cpy["W/L%"] = nba_cpy["W/L%"].astype("float")
    nba_cpy["team"] = nba_cpy["team"].str.replace("\*?\s\(\d{1,2}\)$", "", regex=True)
    team = (
        nba_cpy["team"]
        .str.extract(
            "((?:Los |San |St\. |New |Golden )?(?:\w* )(?:City )?(?P<NBA>.*$))"
        )
        .fillna("")
    )
    nba_cpy["NBA"] = team["NBA"]
    cnt = 0
    cities["NBA"] = cities["NBA"].replace("\[note \d{1,2}\]$", "", regex=True)
    cities.apply(nba_decomp, axis=1)
    cities_cpy["NBA"] = cities_cpy["NBA"].replace("^—$|^$", np.nan, regex=True)
    cities_cpy.dropna(inplace=True)
    cities_cpy["Metropolitan area"] = cities_cpy["Metropolitan area"].replace(
        " Area$", "", regex=True
    )
    cities_nba = cities_cpy[
        ["Metropolitan area", "Population (2016 est.)[8]", "NBA"]
    ].merge(nba_cpy[["NBA", "W/L%"]], on="NBA", how="left")
    wl_ratio = cities_nba.groupby("Metropolitan area", sort=False).agg("mean")["W/L%"]

    population_by_region = pd.unique(cities_nba["Population (2016 est.)[8]"]).astype(
        "float"
    )  # pass in metropolitan area population from cities
    win_loss_by_region = wl_ratio  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(
        win_loss_by_region
    ), "Q2: Your lists must be the same length"
    assert (
        len(population_by_region) == 28
    ), "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [121]:
import re

import numpy as np
import pandas as pd
import scipy.stats as stats

mlb_df = pd.read_csv("assets/mlb.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities_cpy = cities.copy()
cnt = 0


def mlb_decomp(e):
    global cnt
    global cities_cpy
    teams = re.findall(
        "(Blue \w+|White \w+|Red \w+|(?!Blue |Red |White )\w+)", e.loc["MLB"]
    )
    if not teams:
        teams = [""]
    for team in teams:
        try:
            cities_cpy.iloc[cnt] = e
        except:
            cities_cpy = cities_cpy.append(e, ignore_index=True)
        cities_cpy.iloc[cnt].loc["MLB"] = team
        cnt += 1


def mlb_correlation():
    global mlb_df
    global cities_cpy
    global cnt
    cities_cpy = cities.copy()
    nba_cpy = mlb_df[mlb_df["year"] == 2018].copy()
    nba_cpy["W/L%"] = nba_cpy["W-L%"].astype("float")
    team = (
        nba_cpy["team"]
        .str.extract(
            "((?:Los |San |St\. |New )?(?:\w* )(?:Bay )?(?:City )?(?P<MLB>.*$))"
        )
        .fillna("")
    )
    nba_cpy["MLB"] = team["MLB"]
    cnt = 0
    cities["MLB"] = cities["MLB"].replace("\[note \d{1,2}\]$", "", regex=True)
    cities.apply(mlb_decomp, axis=1)
    cities_cpy["MLB"] = cities_cpy["MLB"].replace("^—$|^$", np.nan, regex=True)
    cities_cpy.dropna(inplace=True)
    cities_cpy["Metropolitan area"] = cities_cpy["Metropolitan area"].replace(
        " Area$", "", regex=True
    )
    cities_nba = cities_cpy[
        ["Metropolitan area", "Population (2016 est.)[8]", "MLB"]
    ].merge(nba_cpy[["MLB", "W/L%"]], on="MLB", how="left")
    wl_ratio = cities_nba.groupby("Metropolitan area", sort=False).agg("mean")["W/L%"]

    population_by_region = pd.unique(cities_nba["Population (2016 est.)[8]"]).astype(
        "float"
    )  # pass in metropolitan area population from cities
    win_loss_by_region = wl_ratio  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(
        win_loss_by_region
    ), "Q3: Your lists must be the same length"
    assert (
        len(population_by_region) == 26
    ), "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [122]:
import re

import numpy as np
import pandas as pd
import scipy.stats as stats

nfl_df = pd.read_csv("assets/nfl.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities_cpy = cities.copy()
cnt = 0


def nfl_decomp(e):
    global cnt
    global cities_cpy
    teams = e.loc["NFL"].split()
    if not teams:
        teams = [""]
    for team in teams:
        try:
            cities_cpy.iloc[cnt] = e
        except:
            cities_cpy = cities_cpy.append(e, ignore_index=True)
        cities_cpy.iloc[cnt].loc["NFL"] = team
        cnt += 1


def nfl_correlation():
    global nfl_df
    global cities_cpy
    global cnt
    cities_cpy = cities.copy()
    nhl_cpy = nfl_df[nfl_df["year"] == 2018].copy()
    nhl_cpy.drop(index=[0, 5, 10, 15, 20, 25, 30, 35], inplace=True)
    nhl_cpy["WL%"] = nhl_cpy["W-L%"].astype("float")
    nhl_cpy["team"] = nhl_cpy["team"].str.replace("[*+]|\.{3}", "", regex=True)
    team = (
        nhl_cpy["team"]
        .str.extract(
            "((?:Los |San |St\. |New )?(?:\w* )(?:Bay )?(?:City )?(?P<NFL>.*$))"
        )
        .fillna("")
    )
    nhl_cpy["NFL"] = team["NFL"]
    cnt = 0
    cities["NFL"] = cities["NFL"].replace("\[note \d{1,2}\]$", "", regex=True)
    cities.apply(nfl_decomp, axis=1)
    cities_cpy["NFL"] = cities_cpy["NFL"].replace("^—$|^$", np.nan, regex=True)
    cities_cpy.dropna(inplace=True)
    cities_cpy["Metropolitan area"] = cities_cpy["Metropolitan area"].replace(
        " Area$", "", regex=True
    )
    cities_nhl = cities_cpy[
        ["Metropolitan area", "Population (2016 est.)[8]", "NFL"]
    ].merge(nhl_cpy[["NFL", "WL%"]], on="NFL", how="left")
    wl_ratio = cities_nhl.groupby("Metropolitan area", sort=False)["WL%"].agg("mean")

    population_by_region = pd.unique(cities_nhl["Population (2016 est.)[8]"]).astype(
        "float"
    )  # pass in metropolitan area population from cities
    win_loss_by_region = wl_ratio.astype(
        "float"
    )  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(
        win_loss_by_region
    ), "Q4: Your lists must be the same length"
    assert (
        len(population_by_region) == 29
    ), "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [125]:
def nfl_ratio():
    global nfl_df
    global cities_cpy
    global cities
    global cnt
    cities_cpy = cities.copy()
    nhl_cpy = nfl_df[nfl_df["year"] == 2018].copy()
    nhl_cpy.drop(index=[0, 5, 10, 15, 20, 25, 30, 35], inplace=True)
    nhl_cpy["WL%"] = nhl_cpy["W-L%"].astype("float")
    nhl_cpy["team"] = nhl_cpy["team"].str.replace("[*+]|\.{3}", "", regex=True)
    team = (
        nhl_cpy["team"]
        .str.extract(
            "((?:Los |San |St\. |New )?(?:\w* )(?:Bay )?(?:City )?(?P<NFL>.*$))"
        )
        .fillna("")
    )
    nhl_cpy["NFL"] = team["NFL"]
    cnt = 0
    cities["NFL"] = cities["NFL"].replace("\[note \d{1,2}\]$", "", regex=True)
    cities.apply(nfl_decomp, axis=1)
    cities_cpy["NFL"] = cities_cpy["NFL"].replace("^—$|^$", np.nan, regex=True)
    cities_cpy["Metropolitan area"] = cities_cpy["Metropolitan area"].replace(
        " Area$", "", regex=True
    )
    cities_nhl = cities_cpy[["Metropolitan area", "NFL"]].merge(
        nhl_cpy[["NFL", "WL%"]], on="NFL", how="left"
    )
    return cities_nhl.groupby("Metropolitan area", sort=False)["WL%"].agg("mean")


def nhl_ratio():
    global nhl_df
    global cities_cpy
    global cities
    global cnt
    cities_cpy = cities.copy()
    nhl_cpy = nhl_df[nhl_df["year"] == 2018].copy()
    nhl_cpy.drop(index=[0, 9, 18, 26], inplace=True)
    nhl_cpy["WL%"] = nhl_cpy["W"].astype("float") / (
        nhl_cpy["W"].astype("float") + nhl_cpy["L"].astype("float")
    )
    nhl_cpy["team"] = nhl_cpy["team"].str.replace("\*$", "", regex=True)
    team = (
        nhl_cpy["team"]
        .str.extract("((?:Los |San |St\. |New )?(?:\w* )(?:Bay )?(?P<NHL>.*$))")
        .fillna("")
    )
    nhl_cpy["NHL"] = team["NHL"]
    cnt = 0
    cities["NHL"] = cities["NHL"].replace("\[note \d{1,2}\]$", "", regex=True)
    cities.apply(nhl_decomp, axis=1)
    cities_cpy["NHL"] = cities_cpy["NHL"].replace("^—$|^$", np.nan, regex=True)
    cities_cpy["Metropolitan area"] = cities_cpy["Metropolitan area"].replace(
        " Area$", "", regex=True
    )
    cities_nhl = cities_cpy[
        ["Metropolitan area", "Population (2016 est.)[8]", "NHL"]
    ].merge(nhl_cpy[["NHL", "WL%"]], on="NHL", how="left")
    return cities_nhl.groupby("Metropolitan area", sort=False).agg("mean")["WL%"]


def nba_ratio():
    global nba_df
    global cities_cpy
    global cities
    global cnt
    cities_cpy = cities.copy()
    nba_cpy = nba_df[nba_df["year"] == 2018].copy()
    nba_cpy["W/L%"] = nba_cpy["W/L%"].astype("float")
    nba_cpy["team"] = nba_cpy["team"].str.replace("\*?\s\(\d{1,2}\)$", "", regex=True)
    team = (
        nba_cpy["team"]
        .str.extract(
            "((?:Los |San |St\. |New |Golden )?(?:\w* )(?:City )?(?P<NBA>.*$))"
        )
        .fillna("")
    )
    nba_cpy["NBA"] = team["NBA"]
    cnt = 0
    cities["NBA"] = cities["NBA"].replace("\[note \d{1,2}\]$", "", regex=True)
    cities.apply(nba_decomp, axis=1)
    cities_cpy["NBA"] = cities_cpy["NBA"].replace("^—$|^$", np.nan, regex=True)
    cities_cpy["Metropolitan area"] = cities_cpy["Metropolitan area"].replace(
        " Area$", "", regex=True
    )
    cities_nba = cities_cpy[
        ["Metropolitan area", "Population (2016 est.)[8]", "NBA"]
    ].merge(nba_cpy[["NBA", "W/L%"]], on="NBA", how="left")
    return cities_nba.groupby("Metropolitan area", sort=False).agg("mean")["W/L%"]


def mlb_ratio():
    global mlb_df
    global cities_cpy
    global cities
    global cnt
    cities_cpy = cities.copy()
    nba_cpy = mlb_df[mlb_df["year"] == 2018].copy()
    nba_cpy["W/L%"] = nba_cpy["W-L%"].astype("float")
    team = (
        nba_cpy["team"]
        .str.extract(
            "((?:Los |San |St\. |New )?(?:\w* )(?:Bay )?(?:City )?(?P<MLB>.*$))"
        )
        .fillna("")
    )
    nba_cpy["MLB"] = team["MLB"]
    cnt = 0
    cities["MLB"] = cities["MLB"].replace("\[note \d{1,2}\]$", "", regex=True)
    cities.apply(mlb_decomp, axis=1)
    cities_cpy["MLB"] = cities_cpy["MLB"].replace("^—$|^$", np.nan, regex=True)
    cities_cpy["Metropolitan area"] = cities_cpy["Metropolitan area"].replace(
        " Area$", "", regex=True
    )
    cities_nba = cities_cpy[
        ["Metropolitan area", "Population (2016 est.)[8]", "MLB"]
    ].merge(nba_cpy[["MLB", "W/L%"]], on="MLB", how="left")
    return cities_nba.groupby("Metropolitan area", sort=False).agg("mean")["W/L%"]


def test(s):
    global ratios
    ser = pd.Series(
        [
            stats.ttest_rel(
                ratios[[s.name, index]].dropna()[s.name],
                ratios[[s.name, index]].dropna()[index],
            )[1]
            if index != s.name
            else np.nan
            for index in s.index
        ]
    )
    ser.index = s.index
    return ser

In [126]:
import re

import numpy as np
import pandas as pd
import scipy.stats as stats

mlb_df = pd.read_csv("assets/mlb.csv")
nhl_df = pd.read_csv("assets/nhl.csv")
nba_df = pd.read_csv("assets/nba.csv")
nfl_df = pd.read_csv("assets/nfl.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
ratios = pd.DataFrame(
    data={
        "NHL": nhl_ratio(),
        "NBA": nba_ratio(),
        "NFL": nfl_ratio(),
        "MLB": mlb_ratio(),
    },
    index=cities["Metropolitan area"].replace(" Area$", "", regex=True),
    dtype="float",
)


def sports_team_performance():
    sports = ["NFL", "NBA", "NHL", "MLB"]
    p_values = pd.DataFrame({k: np.nan for k in sports}, index=sports)
    p_values = p_values.apply(test, axis=1)

    assert (
        abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2
    ), "The NBA-NHL p-value should be around 0.02"
    assert (
        abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2
    ), "The MLB-NFL p-value should be around 0.80"
    return p_values